<a href="https://colab.research.google.com/github/vyng0/CS5530_Assignments/blob/main/CS5530_Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
df.to_csv('/content/drive/MyDrive/train.csv', index=False)
df = pd.read_csv('/content/drive/MyDrive/train.csv')

In [65]:
import pandas as pd

df = pd.read_csv("train.csv")
df.head()


,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50


In [66]:
# a) Look for the missing values in all the columns and either impute them
# Check for missing values
df.isnull().sum()


,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36


In [67]:
# (b)Remove the units from the attributes Mileage, Engine, and Power using regular expressions to extract only numeric values.
# This step needs to be done first to prepare the data for imputing missing values in part (a).
# Use astype(str) to ensures every value in the Mileage column is treated as a string
# .str.extract(r'([\d\.]+)') to pull out the numeric part
# astype(float)to converts the extracted string into a float.
df['Mileage'] = df['Mileage'].astype(str).str.extract(r'([\d\.]+)').astype(float)
df['Engine'] = df['Engine'].astype(str).str.extract(r'([\d\.]+)').astype(float)
df['Power'] = df['Power'].astype(str).str.extract(r'([\d\.]+)').astype(float)
df['New_Price'] = df['New_Price'].astype(str).str.replace(' Lakh', '', regex=False)
df['New_Price'] = pd.to_numeric(df['New_Price'], errors='coerce')

# Convert Seats to numeric
df['Seats'] = pd.to_numeric(df['Seats'], errors='coerce')

In [31]:
# (a) Fill in missing values after removing the unit suffixes
# Replace missing values for Mileage with mean (continuous variable)
df['Mileage'] = df['Mileage'].fillna(df['Mileage'].mean())
# Replace missing values for Engine with mean
df['Engine'] = df['Engine'].fillna(df['Engine'].mean())
# Replace missing values for Power with median since Power may have high-end values
df['Power'] = df['Power'].fillna(df['Power'].median())
# Replace missing values for Seats with median (discete variable)
df['Seats'] = df['Seats'].fillna(df['Seats'].median())

# Drop New_Price column since the assignment is not focusing on the new price values
# also there are over 5000 missing values, I chose to drop this column to ensure cleaner and more reliable analysis.
df.drop(columns=['New_Price'], inplace=True)


In [69]:
# Check for missing values
df.isnull().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36


In [70]:
#(c) One-hot encode Fuel_Type and Transmission
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)

# Use pd.get_dummies() with drop_first=True to create new binary columns
## like Fuel_Type_Petrol, Fuel_Type_Electric and Transmission_Manual,
## #while automatically dropping the first category in each case to prevent multicollinearity.


In [71]:
df.columns.tolist()


['Unnamed: 0',
 'Name',
 'Location',
 'Year',
 'Kilometers_Driven',
 'Owner_Type',
 'Mileage',
 'Engine',
 'Power',
 'Seats',
 'New_Price',
 'Price',
 'Fuel_Type_Electric',
 'Fuel_Type_Petrol',
 'Transmission_Manual']

In [72]:
# (d) Create one more feature and add this column to the dataset
# Calculate the car age
from datetime import datetime

# Get current year
current_year = datetime.now().year

# Create a new feature: Car Age
df['Car_Age'] = current_year - df['Year']

df.head()



,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,NaN,12.50,False,False,True,10
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,False,True,True,14
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,NaN,6.00,False,False,True,13
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,NaN,17.74,False,False,False,12
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,NaN,3.50,False,False,True,12


In [78]:
# (e) Perform select, filter, rename, mutate, arrange and summarize with group by operations (or
# their equivalent operations in python) on this dataset. (4 points)

# Select columns Name, Location, Car_Age, and Price
selected_df = df[['Name', 'Location', 'Car_Age', 'Price']]
selected_df.head()


,Name,Location,Car_Age,Price
0,Hyundai Creta 1.6 CRDi SX Option,Pune,10,12.50
1,Honda Jazz V,Chennai,14,4.50
2,Maruti Ertiga VDI,Chennai,13,6.00
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,12,17.74
4,Nissan Micra Diesel XV,Jaipur,12,3.50


In [79]:
# Filter rows of which cars are over 5 years and have more than 5 seats
filtered_df = df[(df['Car_Age'] > 5) & (df['Seats'] > 5)]
filtered_df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,NaN,6.0,False,False,True,13
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755.0,171.50,8.0,21.00,17.5,False,False,False,9
13,15,Mitsubishi Pajero Sport 4X4,Delhi,2014,110000,First,13.50,2477.0,175.56,7.0,32.01,15.0,False,False,True,11
27,29,Toyota Innova 2.5 V Diesel 7-seater,Mumbai,2007,262000,Fourth & Above,12.80,2494.0,102.00,7.0,NaN,4.0,False,False,True,18
30,32,Mahindra Ssangyong Rexton RX7,Bangalore,2013,72000,Second,11.18,2696.0,184.00,7.0,NaN,12.5,False,False,False,12


In [80]:
# Rename column
df.rename(columns={'Name': 'Car_Name'}, inplace=True)

df.columns.tolist()


['Unnamed: 0',
 'Car_Name',
 'Location',
 'Year',
 'Kilometers_Driven',
 'Owner_Type',
 'Mileage',
 'Engine',
 'Power',
 'Seats',
 'New_Price',
 'Price',
 'Fuel_Type_Electric',
 'Fuel_Type_Petrol',
 'Transmission_Manual',
 'Car_Age']

In [81]:
# Mutate - Create feature Mileage_per_Year to calculate how much the car was driver per year
# Avoid dividing by zero or negative age
df['Car_Age'] = df['Car_Age'].apply(lambda x: x if x > 0 else 1)

# Create the feature
df['Mileage_per_Year'] = df['Kilometers_Driven'] / df['Car_Age']

df['Mileage_per_Year'] = df['Mileage_per_Year'].round(3)


df.head()

,Unnamed: 0,Car_Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age,Mileage_per_Year
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,NaN,12.50,False,False,True,10,4100.000
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,8.61,4.50,False,True,True,14,3285.714
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,NaN,6.00,False,False,True,13,6692.308
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,NaN,17.74,False,False,False,12,3389.167
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,NaN,3.50,False,False,True,12,7249.917


In [82]:
# Arrange - Sort cars by price in descending order
sorted_df = df.sort_values(by='Price', ascending=False)
sorted_df.head()


,Unnamed: 0,Car_Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age,Mileage_per_Year
3952,4079,Land Rover Range Rover 3.0 Diesel LWB Vogue,Hyderabad,2017,25000,First,13.33,2993.0,255.0,5.0,NaN,160.00,False,False,False,8,3125.000
5620,5781,Lamborghini Gallardo Coupe,Delhi,2011,6500,Third,6.40,5204.0,560.0,2.0,NaN,120.00,False,True,False,14,464.286
5752,5919,Jaguar F Type 5.0 V8 S,Hyderabad,2015,8000,First,12.50,5000.0,488.1,2.0,NaN,100.00,False,True,False,10,800.000
1457,1505,Land Rover Range Rover Sport SE,Kochi,2019,26013,First,12.65,2993.0,255.0,5.0,NaN,97.07,False,False,False,6,4335.500
1917,1974,BMW 7 Series 740Li,Coimbatore,2018,28060,First,12.05,2979.0,320.0,5.0,NaN,93.67,False,True,False,7,4008.571


In [83]:
# Summarize with Group By
# Summarize the average price per car name (or model)

avg_price_by_name = df.groupby('Car_Name')['Price'].mean().reset_index()
avg_price_by_name.rename(columns={'Price': 'Average_Price'}, inplace=True)

# List top 10
avg_price_by_name.sort_values(by='Average_Price', ascending=False).head(10)



,Car_Name,Average_Price
783,Land Rover Range Rover 3.0 Diesel LWB Vogue,160.000
763,Lamborghini Gallardo Coupe,120.000
742,Jaguar F Type 5.0 V8 S,100.000
128,BMW 7 Series 740Li,93.335
1283,Mercedes-Benz SLK-Class 55 AMG,90.000
792,Land Rover Range Rover Sport SE,80.910
1236,Mercedes-Benz GLS 350d Grand Edition,79.000
1213,Mercedes-Benz E-Class E 350 d,78.800
752,Jaguar XJ 3.0L Portfolio,77.000
751,Jaguar XJ 2.0L Portfolio,75.000
